In [1]:
import os

ASSETS_DIR = os.path.join(os.path.dirname(os.path.abspath(__name__)), 'assets')
INPUTS_DIR = os.path.join(ASSETS_DIR, 'inputs')
OUTPUTS_DIR = os.path.join(ASSETS_DIR, 'outputs')
PICKLES_DIR = os.path.join(ASSETS_DIR, 'pickles')


In [2]:
import re
def text_normalize(text):
    text = str(text)
    # denoise
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ   | # Tatwil/Kashida
                             "  # quotations 
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)
    # text normalizer
    text = re.sub(r"أ|إ|آ", "ا", text)
    text = re.sub(r"ء|ئ|ؤ", "ء", text)
    text = re.sub(r"ى|ي", "ي", text)
    text = re.sub(r"ه|ة", "ه", text)

    # number noramlize
    text = re.sub(r"\d+", "NUM", text)

    # remove non letters
    text = re.sub(r"\W", " ", text)
    
    # if the word is very small neglect it
    text = " ".join([word for word in text.split() if len(word) >= 3 ])

    return text

In [3]:
import pandas as pd
dataset = pd.DataFrame(columns=['text', 'label'])
for file in os.listdir(INPUTS_DIR):
    if not file.startswith('.'):
        temp_df = pd.read_csv(os.path.join(INPUTS_DIR, file))
        # apply normalizing
        temp_df['text'] = temp_df['text'].apply(text_normalize)
        text = " ".join(temp_df['text'].values)
        label = file.split(".")[0]
        dataset = dataset.append({
            "text": text,
            "label": label
        }, ignore_index = True) 
print(dataset['label'].value_counts())
dataset.head()

Sports       1
Education    1
Art          1
Medicine     1
Name: label, dtype: int64


,text,label
0,نصر محروس انتظروا اغاني جديده مصطفي شوقي وسومه...,Art
1,تعلم الطالب رياض الاطفال وحتي الصف الثالث الاب...,Education
2,اطمن علي نفسك فحص البروستاتا ومتي يجب اجراءه n...,Medicine
3,NUM مباراه جمعت الزمالك والمقاولون اللقاء الاو...,Sports


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2), sublinear_tf=True)

In [5]:
vectorizer.fit(dataset['text'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=True, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [6]:
tf_idfs = pd.DataFrame(vectorizer.transform(dataset['text']).toarray(), columns=vectorizer.get_feature_names())
tf_idfs.to_csv("tf-idfs.csv")

In [7]:
tf_idfs[tf_idfs>0.01].iloc[0, :]

academy             NaN
academy بمغامرات    NaN
act                 NaN
act international   NaN
act org             NaN
                     ..
ﺷﺒهه اميتاب         NaN
ﺷﻜﺮي                NaN
ﺷﻜﺮي ﺳﺮﺣﺎن          NaN
ﻋﻤﺮي                NaN
ﻋﻤﺮي ھﺒﻘﻰ           NaN
Name: 0, Length: 123621, dtype: float64

In [8]:
tf_idfs = {}
for label in dataset['label'].unique():
    tf_idfs[label] = vectorizer.transform(dataset.loc[dataset['label'] == label]['text'])

In [9]:
from sklearn.metrics.pairwise import linear_kernel
import operator

def classify(text):
    text = text_normalize(text)
    text_v = vectorizer.transform([text])
    distances = {}
    for key in tf_idfs.keys():
        distances[key] = linear_kernel(text_v, tf_idfs[key]).flatten()[0]
    return distances

In [11]:
texts = [
    "ما توقعاتك لنتيجة لقاء #الأهلي و #غزل_المحلة في الدوري المصري اليوم؟ تنطلق في السابعة والنصف مساء",
    "لأهلي يستقر على تشكيل مواجهة غزل المحلة فى الدوري",
    "أسرع انتشارًا.. رصد سلالة جديدة من كورونا فى بريطانيا",
    "برازيلية تحول منزلها لمستشفى دمى لإسعاد الأطفال فى زمن الكورونا",
    "شوبير يعلن سلبية مسحة ثنائى منتخب الشباب وانفراجة لخوض مواجهة تونس",
    "لوسى تودع 2020 بوصلة رقص على أغنية تامر عاشور باى باى.. فيديو وصور",
    "إلهام شاهين تشكر ليوم السابع بعد ندوة الاحتفاء بتكريمها فى مهرجان القاهرة",
    "شاهد رقصة مع آسر ياسين وموقف طريف فى عيد ميلاد نيللى كريم",
    "حزب الغد لوزير التعليم نطالب بامتحان واحد شامل هذا العام والغاء امتحان نصف الترم",
    "عاجل : وزير التعليم : اليوم أكبر نسبة حضور في العام الدراسي ولابد من استكمال الدراسة",
    "أول نسخة ضوئية من قرارات وزير التعليم برفع الغياب ونظام التقويم"
]
for text in texts:
    distances = classify(text)
    #print("for text input : {} ditances are :".format(text))
    #print(sorted(distances.items(), key=operator.itemgetter(1), reverse=True))
    #print()
    print("for text input:\n {}".format(text))
    print(sorted(distances.items(), key=operator.itemgetter(1), reverse=True)[0][0])
    print()

for text input:
 ما توقعاتك لنتيجة لقاء #الأهلي و #غزل_المحلة في الدوري المصري اليوم؟ تنطلق في السابعة والنصف مساء
Sports

for text input:
 لأهلي يستقر على تشكيل مواجهة غزل المحلة فى الدوري
Sports

for text input:
 أسرع انتشارًا.. رصد سلالة جديدة من كورونا فى بريطانيا
Medicine

for text input:
 برازيلية تحول منزلها لمستشفى دمى لإسعاد الأطفال فى زمن الكورونا
Medicine

for text input:
 شوبير يعلن سلبية مسحة ثنائى منتخب الشباب وانفراجة لخوض مواجهة تونس
Sports

for text input:
 لوسى تودع 2020 بوصلة رقص على أغنية تامر عاشور باى باى.. فيديو وصور
Art

for text input:
 إلهام شاهين تشكر ليوم السابع بعد ندوة الاحتفاء بتكريمها فى مهرجان القاهرة
Art

for text input:
 شاهد رقصة مع آسر ياسين وموقف طريف فى عيد ميلاد نيللى كريم
Art

for text input:
 حزب الغد لوزير التعليم نطالب بامتحان واحد شامل هذا العام والغاء امتحان نصف الترم
Education

for text input:
 عاجل : وزير التعليم : اليوم أكبر نسبة حضور في العام الدراسي ولابد من استكمال الدراسة
Education

for text input:
 أول نسخة ضوئية من قرارات وزير التع